In [8]:
import numpy as np
import pyopencl as cl
import pyopencl.array

In [3]:
%load_ext pyopencl.ipython_ext

In [4]:
ctx = cl.create_some_context(interactive=True)

In [5]:
queue = cl.CommandQueue(ctx)

In [6]:
%%cl_kernel -o "-cl-fast-relaxed-math"

__kernel void sum_vector(__global const float *a,__global const float *b, __global float *c)
{
    int gid = get_global_id(0);
    c[gid] = a[gid] + b[gid];
}

In [9]:
n = 10000

a = cl.array.empty(queue, n, dtype=np.float32)
a.fill(15)

b_host = np.random.randn(n).astype(np.float32)
b = cl.array.to_device(queue,b_host)

c = cl.array.empty_like(a)

In [11]:
sum_vector(queue, (n,), None, a.data, b.data, c.data)

In [12]:
c.get()

array([ 14.95187092,  15.87479401,  15.54437542, ...,  14.49657345,
        14.83622551,  14.66652966], dtype=float32)

In [14]:
(c.get() == b_host + 15).all()

True

In [21]:
p = cl.get_platforms()[0]
p.get_devices()

[<pyopencl.Device 'Intel(R) Core(TM) i7-4770HQ CPU @ 2.20GHz' on 'Apple' at 0xffffffff>,
 <pyopencl.Device 'Iris Pro' on 'Apple' at 0x1024500>]

In [26]:
p.get_devices()[0].name

'Intel(R) Core(TM) i7-4770HQ CPU @ 2.20GHz'

In [27]:
p.get_devices()[1].name

'Iris Pro'